# FAISS

Faiss (Facebook AI Similarity Search) — это библиотека с открытым исходным кодом, разработанная Facebook AI, предназначенная для эффективного поиска и кластеризации данных в больших наборах векторов. Она оптимизирована для обработки больших массивов данных и часто используется в задачах, связанных с машинным обучением, особенно для поиска ближайших соседей (Nearest Neighbor Search, NNS).

То есть идея - быстро находить самые близкие векторы

### Основные применения Faiss

    Поиск ближайших соседей (ANN):
        Поиск похожих векторов в больших наборах данных.
        Используется в рекомендательных системах, системах поиска, кластеризации данных.

    Рекомендательные системы:
        Поиск похожих пользователей, товаров или контента на основе векторных представлений.

    Семантический поиск:
        Использование текстовых эмбеддингов для поиска документов, вопросов или ответов.

    Кластеризация данных:
        Эффективная обработка больших наборов векторов для сегментации.

    Интеграция с NLP и CV задачами:
        Семантические эмбеддинги из трансформеров (например, BERT) или фичи из сверточных нейронных сетей.

In [23]:
!pip install faiss-cpu
# или для GPU
#pip install faiss-gpu


## Популярные индексы в Faiss

- **Flat (IndexFlatL2, IndexFlatIP)**:
    - Полный перебор всех элементов.
    - Очень точный, но медленный для больших данных.

- **IVF (IndexIVFFlat, IndexIVFPQ)**:
    - Разделяет пространство векторов на кластеры.
    - Балансирует между скоростью и точностью.

- **HNSW (IndexHNSW)**:
    - Структура графа для быстрого поиска.
    - Высокая производительность и гибкость.

- **PQ (Product Quantization)**:
    - Используется для сильного уменьшения памяти.
    - Часто используется для обработки миллиардов объектов.

### Замечание

Тут везде работа с евклидовым расстоянием, но если все вектора нормированы то

$$
cosine \space similarity = 1 - \frac{euclidian \space distance^2}{2}
$$

то есть близость по L<sub>2</sub> означает и близость косинусовых расстояний
 

### Flat

In [24]:
import faiss
import numpy as np

# Генерируем случайные векторы (1000 объектов размерностью 128)
d = 8  # Размерность векторов
nb = 1000  # Количество векторов в базе
np.random.seed(1234)

data = np.random.random((nb, d)).astype('float32')

# Создаем индекс для поиска
index = faiss.IndexFlatL2(d)  # L2 — евклидово расстояние
print("Is trained:", index.is_trained)  # Flat-индексы всегда "обучены"

# Добавляем данные в индекс
index.add(data)
print("Количество векторов в индексе:", index.ntotal)

# Поиск ближайших соседей
query = np.random.random((2, d)).astype('float32')  # 5 запросов
print("Вектора-запросы:\n",query[:2])

D, I = index.search(query, k=3)  # Находим 3 ближайших соседа

print("Индексы ближайших соседей:\n", I)
print("Вектора-соседи:\n", np.take(data, I[:2], axis=0))

print("Расстояния до них:\n", D)

Is trained: True
Количество векторов в индексе: 1000
Вектора-запросы:
 [[0.93991697 0.38034382 0.51122963 0.30554125 0.14744776 0.59094244
  0.35732955 0.20288098]
 [0.75160396 0.7827614  0.10776526 0.62013817 0.02783209 0.19797945
  0.9863211  0.7784719 ]]
Индексы ближайших соседей:
 [[431 412 121]
 [158 990 285]]
Вектора-соседи:
 [[[0.8867884  0.5634721  0.29832563 0.36154348 0.02120025 0.76184565
   0.27947828 0.27432093]
  [0.90471286 0.43641537 0.396005   0.04543002 0.14193672 0.46587282
   0.57073486 0.19884548]
  [0.84769595 0.5296996  0.40959972 0.01830795 0.14865962 0.8043352
   0.41977814 0.33921757]]

 [[0.7527404  0.8047769  0.2944434  0.780619   0.2247863  0.28984916
   0.84898186 0.74416554]
  [0.97722316 0.9228401  0.42988458 0.71233296 0.13372402 0.03489907
   0.8702576  0.88712096]
  [0.7856904  0.8615666  0.42937538 0.6854403  0.26476413 0.1950038
   0.8205175  0.5202842 ]]]
Расстояния до них:
 [[0.1411338  0.14654882 0.1916689 ]
 [0.1283588  0.24587047 0.26536697]]


### HNSW

(Hierarchical Navigable Small World) 

In [25]:
import faiss
import numpy as np

# Генерируем случайные векторы (1000 объектов размерностью 128)
d = 8  # Размерность векторов
nb = 1000  # Количество векторов в базе
np.random.seed(1234)
data = np.random.random((nb, d)).astype('float32')

# Создаем HNSW индекс
M = 32  # Количество соседей в графе (больше M — лучше точность, но выше память)
index = faiss.IndexHNSWFlat(d, M)  # L2 расстояние используется по умолчанию
index.hnsw.efConstruction = 40  # Параметр управления качеством построения графа

# Добавляем данные в индекс
index.add(data)
print("Количество векторов в индексе:", index.ntotal)

# Увеличиваем efSearch для повышения точности поиска
index.hnsw.efSearch = 16  # Чем больше значение, тем выше точность, но медленнее поиск

# Создаем запросы (2 случайных векторов)
query = np.random.random((2, d)).astype('float32')
print("Вектора-запросы:\n",query[:2])

# Поиск ближайших соседей
k = 3  # Количество ближайших соседей
D, I = index.search(query, k)  # Возвращает расстояния (D) и индексы (I)

# Вывод результатов
print("Индексы ближайших соседей:\n", I)
print("Расстояния до них:\n", D)

# Извлечение самих векторов ближайших соседей
neighbors = np.take(data, I, axis=0)
print("Вектора ближайших соседей:\n", neighbors)

# Для каждого запроса можно посмотреть его соседей
for query_idx, (indices, vectors) in enumerate(zip(I, neighbors)):
    print(f"Запрос {query_idx + 1}:")
    print(f"Индексы соседей: {indices}")
    print(f"Вектора соседей:\n{vectors}\n")

Количество векторов в индексе: 1000
Вектора-запросы:
 [[0.93991697 0.38034382 0.51122963 0.30554125 0.14744776 0.59094244
  0.35732955 0.20288098]
 [0.75160396 0.7827614  0.10776526 0.62013817 0.02783209 0.19797945
  0.9863211  0.7784719 ]]
Индексы ближайших соседей:
 [[431 412 121]
 [158 990 285]]
Расстояния до них:
 [[0.1411338  0.14654882 0.1916689 ]
 [0.1283588  0.24587047 0.26536697]]
Вектора ближайших соседей:
 [[[0.8867884  0.5634721  0.29832563 0.36154348 0.02120025 0.76184565
   0.27947828 0.27432093]
  [0.90471286 0.43641537 0.396005   0.04543002 0.14193672 0.46587282
   0.57073486 0.19884548]
  [0.84769595 0.5296996  0.40959972 0.01830795 0.14865962 0.8043352
   0.41977814 0.33921757]]

 [[0.7527404  0.8047769  0.2944434  0.780619   0.2247863  0.28984916
   0.84898186 0.74416554]
  [0.97722316 0.9228401  0.42988458 0.71233296 0.13372402 0.03489907
   0.8702576  0.88712096]
  [0.7856904  0.8615666  0.42937538 0.6854403  0.26476413 0.1950038
   0.8205175  0.5202842 ]]]
Запрос 

### (to be continued)